In [1]:
from tqdm import tqdm
import joblib
from collections import Counter

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy import sparse

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
# from umap import UMAP

from sklearn.preprocessing import StandardScaler

from utils import get_DF #, get_data_XY, w2v, train_test_split

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

In [ ]:
# df = get_DF()

In [2]:
load = 237007
load = ""
load = "_0.15"
# load = None

sentence_percent = 0.15

min_df, max_df = 3, 0.7


if load is None:
    with open("data/sents.txt") as handle:
        sents = [s for s in tqdm(handle) if len(s.strip()) > 1]
        
    downsample = np.random.choice(len(sents), size=int(len(sents)*sentence_percent))
    downsample = (sents[i] for i in tqdm(downsample))
    
    print("downsampled")
    tfidf = TfidfVectorizer(max_df=max_df, min_df=min_df) #max_features=100000, 
    mat = tfidf.fit_transform(downsample)
    print("matrix shape", mat.shape)

    tfidf.stop_words_ = None
    joblib.dump(tfidf, "vectors/tfidf_0.15.pkl")
    sparse.save_npz('vectors/tfidf_mat_0.15.npz', mat)
    
else:
    print(f'Loading "vectors/tfidf{load}.pkl"')
    tfidf = joblib.load(f'vectors/tfidf{load}.pkl')
    mat = sparse.load_npz(f'vectors/tfidf_mat{load}.npz')
    print(mat.shape)

Loading "vectors/tfidf_0.15.pkl"
(2614337, 210218)


---
# Dimensionality Reduction

In [3]:
from sklearn.decomposition import PCA, TruncatedSVD


svd = TruncatedSVD(n_components=100)
svdmat = svd.fit_transform(mat.T)

4598195/(871445*94464), 871445*94464

(5.585744556423985e-05, 82320180480)

In [4]:
svdmat.shape, mat.shape, mat.size, (svdmat > 0.00001).sum()/svdmat.size, mat.size/(mat.shape[0]*mat.shape[1])

((210218, 100),
 (2614337, 210218),
 14166050,
 0.44155686002150146,
 2.577610552348156e-05)

In [5]:
len(tfidf.get_feature_names())

210218

In [6]:
np.savetxt("svd_mat_0.15.tsv", svdmat, delimiter="\t")

In [ ]:
from umap import UMAP


umap = UMAP(
        n_neighbors=15, min_dist=0.1, n_components=2, metric='euclidean',
        verbose=True
    )

orig_n = mat.T.shape[0]
rand_inds = np.random.randint(orig_n, size=orig_n//10)
umat = umap.fit_transform(mat.T)